In [1]:
!pip install pywedge

     |████████████████████████████████| 1.9MB 9.2MB/s 
     |████████████████████████████████| 66.3MB 43kB/s 
     |████████████████████████████████| 143kB 51.5MB/s 
     |████████████████████████████████| 112kB 61.4MB/s 
     |████████████████████████████████| 225kB 58.3MB/s 
     |████████████████████████████████| 552kB 48.7MB/s 
  Found existing installation: jupyter-client 5.3.5
    Uninstalling jupyter-client-5.3.5:
      Successfully uninstalled jupyter-client-5.3.5
  Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Importing libraries

In [18]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

### Testing Dataset

In [26]:
test_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/test.csv")

In [27]:
pdf = test_data['Product_id'].values

### Training Dataset

In [28]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/train.csv")
data.shape

(6368, 15)

In [29]:
data.head()

,Product_id,Stall_no,instock_date,Market_Category,Customer_name,Loyalty_customer,Product_Category,Grade,Demand,Discount_avail,charges_1,charges_2 (%),Minimum_price,Maximum_price,Selling_Price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12.000,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13.000,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22.000,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01.000,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53.000,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935


Since, Selling_Price column also contains negative values, we need to take absolute values of those selling prices. 

In [30]:
data['Selling_Price'] = data['Selling_Price'].abs()

Out of 15 columns first fourteen columns are our features and last column is our target column/label.

### Data Preprocessing

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6368 entries, 0 to 6367
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product_id        6368 non-null   object 
 1   Stall_no          6363 non-null   float64
 2   instock_date      6368 non-null   object 
 3   Market_Category   6368 non-null   int64  
 4   Customer_name     6157 non-null   object 
 5   Loyalty_customer  6368 non-null   object 
 6   Product_Category  6368 non-null   object 
 7   Grade             6368 non-null   int64  
 8   Demand            6368 non-null   int64  
 9   Discount_avail    6330 non-null   float64
 10  charges_1         6170 non-null   float64
 11  charges_2 (%)     6163 non-null   float64
 12  Minimum_price     6330 non-null   float64
 13  Maximum_price     6025 non-null   float64
 14  Selling_Price     6327 non-null   float64
dtypes: float64(7), int64(3), object(5)
memory usage: 746.4+ KB


There are many fields where data is missing, so dropping all those fields.

In [32]:
data = data.dropna()
data.shape

(5397, 15)

There are many features which do not have much impact on deciding the selling prices, so dropping all those columns.

In [33]:
data = data.drop(['Product_id','instock_date','Stall_no','Customer_name','Loyalty_customer','Demand','Product_Category'],axis=1)
test_data = test_data.drop(['Product_id','instock_date','Stall_no','Customer_name','Loyalty_customer','Demand','Product_Category'],axis=1)

Using pywedge library for data preprocessing

In [34]:
import pywedge as pw
ppd = pw.Pre_process_data(data, test_data, y='Selling_Price',c=None,type="Regression")

dataframe_clean() method in pywedge helps to clean data inside dataframe, it interactively asks for methods using which we want to preprocess our data.

In [35]:
new_X, new_y, new_test = ppd.dataframe_clean()

Reading the datasets...
******************************************

Train Dataframe summary...
******************************************

Your selected train dataframe has 8 columns and 5397 Rows.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Zero Values, Missing Values, % of Total Values, Total Zero Missing Values, % Total Zero Missing Values, Data Type]
Index: []
******************************************
Starting data cleaning...
******************************************
Do you want to use get_dummies or catcodes to convert categorical to numerical? 
	press 1 for catcodes - Quick info link - https://bit.ly/3lruqtf 
	press 2 for getdummies - Quick info link - https://bit.ly/3d76p7A 
1
Comleted categorical column transformation
******************************************
Do you want to standardize the data? 
	press 1 for Standard Scalar - Quick info link - https://bit.ly/2GPyG6w 
	press 2 for Robust Scalar - Quick info link - https://bit.ly/3jFNCD5 
	press 3

In [36]:
X_train = new_X
y_train = new_y
X_test = new_test

Preparing a baseline model

In [37]:
blm = pw.baseline_model(X_train,y_train)

Regression_summary() provides us best features based on their feature importance scores, also it gives us an idea about different ML algorithms, and how they perform over the dataset. 

In [38]:
blm.Regression_summary()

Starting regression summary...
TOP 10 FEATURE IMPORTANCE TABLE
Maximum_price      0.785476
Discount_avail     0.121125
Minimum_price      0.084280
charges_1          0.007181
Market_Category    0.001073
charges_2 (%)      0.000718
Grade              0.000147
dtype: float64
Please enter test size: (for eg. please enter 0.20 for 20% test size): 
	0.2
--------------------------LINEAR MODELS---------------------------------
Linear Reg     | exp_var = 0.891 | mae = 451.979 | rmse = 21.260 | r2 = 0.890 | Train time = 0.002s | Pred. time = 0.000s
KNN            | exp_var = 0.591 | mae = 1,139.562 | rmse = 33.757 | r2 = 0.590 | Train time = 0.006s | Pred. time = 0.008s
LinearSVR      | exp_var = 0.130 | mae = 3,937.447 | rmse = 62.749 | r2 = -1.550 | Train time = 0.263s | Pred. time = 0.000s
Lasso          | exp_var = 0.890 | mae = 451.713 | rmse = 21.254 | r2 = 0.890 | Train time = 0.002s | Pred. time = 0.000s
Ridge          | exp_var = 0.890 | mae = 451.715 | rmse = 21.254 | r2 = 0.890 | Tra

Here, we can conclude best ML algorithms that we can use on this dataset are Random Forest, GB Regressor, CB Regressor, XGB Regressor. 
I'm using here Gradient Boosting Regressor, XGBoost Regressor and Random Forests. 

In [39]:
X_t,X_v,Y_t,Y_v = train_test_split(X_train.values,y_train.values,test_size=0.2,random_state=42)

### Using Gradient Boosting Regressor

In [40]:
from sklearn.ensemble import GradientBoostingRegressor

In [47]:
gbr = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.2,learning_rate=0.01, loss='ls', max_depth=7,
                          max_features=None, max_leaf_nodes=None, min_impurity_split=0.1,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=15000,
                          random_state=42, subsample=0.7)

In [48]:
gb_model = gbr.fit(X_t,Y_t)

In [49]:
print("Accuracy of training data is: ",gb_model.score(X_t,Y_t))
print("Accuracy of testing data is: ",gb_model.score(X_v,Y_v))

Accuracy of training data is:  0.9999880658329608
Accuracy of testing data is:  0.9993430836166824


In [50]:
GBpredict = gbr.predict(X_v)
GBpredict = np.absolute(GBpredict)

In [125]:
print(np.sqrt(mean_squared_log_error( Y_v,GBpredict )))

0.018855772220086723


In [55]:
gbpredict = gb_model.predict(X_test.values)
arr1 = np.stack((pdf,gbpredict),axis=1)

In [56]:
gbpred_df['Selling_Price'] = gbpred_df['Selling_Price'].abs()
gbpred_df  = pd.DataFrame(arr1,columns=['Product_id','Selling_Price'])

In [59]:
gbpred_df.to_csv("GBPredict.csv",index=False)
from google.colab import files
files.download('GBPredict.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Using Random Forest Regressor

In [60]:
from sklearn.ensemble import RandomForestRegressor

In [61]:
rf = RandomForestRegressor(max_depth=17,max_features=7,max_leaf_nodes=1925,max_samples= 3400,min_impurity_decrease=0.0,
 min_impurity_split= None,min_samples_leaf= 1,min_samples_split= 2,min_weight_fraction_leaf= 0.0,n_estimators=250,
random_state=42)

In [62]:
rf_model = rf.fit(X_t,Y_t)

In [63]:
print("Accuracy of training data is: ",rf_model.score(X_t,Y_t))
print("Accuracy of testing data is: ",rf_model.score(X_v,Y_v))

Accuracy of training data is:  0.9991932738591511
Accuracy of testing data is:  0.9988850032450117


In [64]:
RFpredict = rf_model.predict(X_v)
RFpredict = np.absolute(RFpredict)

In [126]:
print(np.sqrt(mean_squared_log_error( Y_v, RFpredict )))

0.028821403667411104


In [66]:
rfpredict = rf_model.predict(X_test.values)

In [67]:
arr2 = np.stack((pdf,rfpredict),axis=1)
rfpred_df  = pd.DataFrame(arr2,columns=['Product_id','Selling_Price'])
rfpred_df['Selling_Price'] = rfpred_df['Selling_Price'].abs()

In [68]:
rfpred_df.to_csv("RFPredict.csv",index=False)
from google.colab import files
files.download('RFPredict.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Using XGBoost Regressor

In [69]:
from xgboost import XGBRegressor

In [70]:
xgr =XGBRegressor(max_depth=7,reg_alpha=0.005,n_estimators=17000,learning_rate=0.01)

In [71]:
xg_model = xgr.fit(X_t,Y_t)

[13:05:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [72]:
XGpredict = xg_model.predict(X_v)
XGpredict = np.absolute(XGpredict)

In [73]:
print("Accuracy of training data is: ",xg_model.score(X_t,Y_t))
print("Accuracy of testing data is: ",xg_model.score(X_v,Y_v))

Accuracy of training data is:  0.9999999072726595
Accuracy of testing data is:  0.9991610654782287


In [74]:
print(np.sqrt(mean_squared_log_error(Y_v,X_val_predict )))

0.018855772220086723


In [75]:
xgpredict = xg_model.predict(X_test.values)

In [76]:
arr3 = np.stack((pdf,xgpredict),axis=1)
xgpred_df  = pd.DataFrame(arr3,columns=['Product_id','Selling_Price'])
xgpred_df['Selling_Price'] = xgpred_df['Selling_Price'].abs()

In [77]:
xgpred_df.to_csv("XGPredict.csv",index=False)
from google.colab import files
files.download('XGPredict.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Stacked Regression

In [78]:
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler

In [79]:
kfolds = KFold(n_splits=8,shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_log_error(y, y_pred))

def cv_rmse(model, X=X_t):
    rmse = np.sqrt(-cross_val_score(model, X, Y_t, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [80]:
stack_gen = StackingCVRegressor(regressors=(gbr,rf,xgr),
                                meta_regressor=xgr,use_features_in_secondary=True,store_train_meta_features=True)

In [81]:
scores={}

In [82]:
score = cv_rmse(gbr)
print("gradient boost: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), )

score = cv_rmse(xgr)
print("xgboost: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), )

score = cv_rmse(rf)
print("rf: {:.4f} ({:.4f})\n".format(score.mean(), score.std()), )

gradient boost: 147.5888 (75.7807)

[13:27:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:27:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:28:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:29:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:30:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:31:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:32:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:33:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:lin

In [83]:
print('stack_gen')
stack_gen_model = stack_gen.fit(X_t,Y_t)

stack_gen
[13:44:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:45:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:46:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:46:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:47:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:48:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:52:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [215]:
def blend_models(X):
    return ((0.65* gb_model.predict(X)) + \
            #(0.1 * xg_model.predict(X)) + \
            #(0.05 * rf_model.predict(X)) + \
            (0.35 * stack_gen_model.predict(X)))

In [216]:
blend_v = blend_models(X_v)
blend_v = np.absolute(blend_v)

In [217]:
print('RMSLE score on test data:')
print(rmsle(Y_v, blend_v))
print(np.sqrt(mean_squared_error( Y_v,blend_v)))

RMSLE score on test data:
0.018398127381783216
70.58772699150552


In [211]:
blendpredict = blend_models(X_test.values)

In [212]:
new_arr = np.stack((pdf,blendpredict),axis=1)

In [213]:
blendpred_df  = pd.DataFrame(new_arr,columns=['Product_id','Selling_Price'])
blendpred_df['Selling_Price'] = blendpred_df['Selling_Price'].abs()

In [214]:
blendpred_df.to_csv("BP.csv",index=False)
from google.colab import files
files.download('BP.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>